In [1]:
# Required Python packages
from datetime import date, datetime, timedelta
import calendar
import pandas as pd
import numpy as np
import itertools
import copy
from numpy.linalg import multi_dot
from scipy.stats import norm
from scipy.stats import bernoulli
from scipy.optimize import fmin_slsqp as min

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.api import OLS

from sklearn.linear_model import LassoCV
from yellowbrick.datasets import load_concrete
from yellowbrick.regressor import AlphaSelection

import matplotlib.pyplot as plt
import math


#Import Date opertative user defined functions
from ipynb.fs.full.user_defined_vik_functions import get_all_monthly_option_expiries, \
                                                     find_last_thurs_date_of_month, \
                                                     prev_workday_if_holiday, find_wkly_expries,\
                                                     date_of_prev_thurs

# Import dataframe naming functions 
from ipynb.fs.full.user_defined_vik_functions import get_mthly_df_name_from_expiry

#Import data loading functions
from ipynb.fs.full.user_defined_vik_functions import load_all_mthly_data

#import traded options parameters and info
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_strikes, generate_mthly_strikes

#Import pricing functions
from ipynb.fs.full.user_defined_vik_functions import BSM_call_vec_with_div, BSM_put_vec_with_div

#Import implied volatility processing functions
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_iv, gen_bivariate_forward_smile

#Import Risk-free interest rate generation functions
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_ir

#Import Stochatic process simulation functions
from ipynb.fs.full.user_defined_vik_functions import gbm1W_simulation, generate_covariance_from_correlation

# Import risk free interest rate function
from ipynb.fs.full.user_defined_vik_functions import get_risk_free_rate_from_exact_date


In [17]:
# Stock Index of Interest
# stock_ident = "BANKNIFTY"
stock_ident = "NIFTY"

#Static hedging performed at different moneyness regions 
#i.e. moneyness is used to select the option with nearest moneyness match
# ATM - At the Money, ITM - In the money, OTM - Out of the Money
prod_moneyness = "OTM"

#Product type to hedge: either "CE" or "PE"
# prod_type = "CE"
prod_type = "PE"

# The scope of this code is to hedge one option 
# and scope will be extended to a portfolio
no_of_assets = 1
cor_mat = [[1]]

#Path to refer data
source_path = "/home/jupyter-partha/Vikranth - Chapter 2/"
input_sub_path = "Input Data/mkt_data_covid_region/"
output_sub_path = "Output Data/"
input_data_path = source_path + input_sub_path
output_data_path = source_path + output_sub_path

# Periods of interest will be a dictionary
#Key is the year, value is a list of months 1-12, 1- Jan, 2 - Feb,...12 - Dec
# For E.g., periods_of_interest = {2020: [3], 2019: [11, 12]}
periods_of_interest = {2019: [8, 9, 10, 11, 12], 2020: [1, 2, 3, 4, 5, 6, 7]}

#List of holidays
holidays_list = [date(2019, 3, 4), date(2019, 3, 21),\
                 date(2019, 4, 17), date(2019, 4, 19), date(2019, 4, 29),\
                 date(2019, 5, 1),\
                 date(2019, 6, 5),\
                 date(2019, 8, 12), date(2019, 8, 15),\
                 date(2019, 9, 2), date(2019, 9, 10), \
                 date(2019, 10, 2), date(2019, 10, 8), date(2019, 10, 21), date(2019, 10, 28), \
                 date(2019, 11, 12), \
                 date(2019, 12, 25), \
                 date(2020, 2, 21), \
                 date(2020, 3, 10), \
                 date(2020, 4, 2), date(2020, 4, 6), date(2020, 4, 10), date(2020,4, 14), \
                 date(2020, 5, 1), date(2020, 5, 25), \
                 date(2020, 10, 2), date(2020, 11, 16), date(2020, 11, 30), date(2020, 12, 25)]
#Simulation Parameters
no_of_paths = 5000

no_opt=1


In [18]:
#########################
#User-defined functions #
#########################

def generate_weekly_static_pfl_weights(no_of_paths, mthly_expiries_list, \
                                       dict_wkly_expiries_each_month, \
                                       dict_ce_wkly_strikes, dict_pe_wkly_strikes, \
                                       dict_wkly_spots, dict_wkly_iv, \
                                       prod_type, prod_moneyness, output_path, \
                                       option_type = "CE", no_of_assets=1, cor_mat=[[1]],\
                                       stock_ident="BANKNIFTY"):
    ce_pfl_weights = []
    pe_pfl_weights = [] 
    cash_weights = []
    for each_month in mthly_expiries_list:
        strike = dict_mthly_strikes[each_month.strftime("%d-%b-%Y")]
        weekly_pfl_weights = []
        ce_dict = {}
        pe_dict = {}
        cash_dict = {}
        week_list = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")]
        no_of_weeks = len(week_list)
        
        for week in range(0, no_of_weeks):
            s_t = dict_wkly_spots[each_month.strftime("%d-%b-%Y")][week]
            K = strike
            r = np.array([dict_wkly_ir[each_month.strftime("%d-%b-%Y")][week]])
            
            if (week == no_of_weeks-1):
                sim_start_date = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")][week]
                sim_end_date = each_month
            else:
                sim_start_date = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")][week]
                sim_end_date = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")][week + 1]

            df_full_iv = pd.read_csv(output_path + "A3_" + stock_ident + "_implied_vol_surface.csv")
            df_date = df_full_iv[df_full_iv['Date'] == sim_start_date.strftime("%d-%b-%Y")]
            
            # We define Moneyness as (S/K) in vol surface for both calls and puts
            option_moneyness = float(s_t/float(K))
            sim_moneyness = 1.0
            
            # Vol Surface is already sorted when created in code A3 - This is just to make sure again
            df_date = df_date.sort_values(['T', 'Moneyness'], ascending=[False, True]).reset_index()
            
            # np.unique also sorts the array - we need to re-sort in descending order
            # This is required as last week has only one tenor and when filtering, we need to filter as 
            # tenor_list[0]
            
            t_list = np.sort(np.unique(np.array(df_date['T'])))
            tenor_list = [str(int(round(t* 365))) + "D" for t in t_list]
            
            print(each_month, " Week: ", week)
            print(tenor_list)
            
            if (len(tenor_list) > 2):
                print("There are more than 2 tenors in the vol surface - Please check!")
                
            df_tenor = df_date[df_date['Tenor'] == tenor_list[0]]

            x = np.array(df_tenor["Moneyness"])
            y = np.array(df_tenor["Impl_Vol"])
            vol_list = [np.interp(sim_moneyness, x, y)]                
                
            sim_stock_mat = gbm1W_simulation(no_of_paths, sim_start_date, sim_end_date, \
                                             no_of_assets, s_t, r, vol_list, cor_mat)
            
            call_strikes = np.sort(np.array(dict_ce_wkly_strikes[each_month.strftime("%d-%b-%Y")][week])).reshape(1,-1)
            put_strikes = np.sort(np.array(dict_pe_wkly_strikes[each_month.strftime("%d-%b-%Y")][week])).reshape(1,-1)
            no_of_calls = call_strikes.size
            no_of_puts = put_strikes.size
            no_ind_vars = no_of_calls + no_of_puts
                      
            stock_vec = sim_stock_mat[:, 1]
            r_f = get_risk_free_rate_from_exact_date(sim_start_date)
            dt = float((each_month - sim_end_date).days) / 365
            
            opt_strike = strike
            
            spot_list = sim_stock_mat[:,1]
            mny_list = spot_list / K
            

            # Finding Forwrd Vol in varaince space across tenors
            if (week != no_of_weeks-1):
                
                # Time variables
                t1 = t_list[0]
                t2 = t_list[1] 
                t12 = dt

                # Vol at larger tenor: corresponding to maturity of option to be hedged
                df_tenor = df_date[df_date['Tenor'] == tenor_list[1]]
                x = np.array(df_tenor["Moneyness"])
                y = np.array(df_tenor["Impl_Vol"])
                
                vol_t2 = np.interp(mny_list, x, y)

                #Variance corresponding to tenor t2
                var_t2 = np.square(vol_t2)
                
                #Vol at weekly tenor: corresponding to maturity of options used to hedge
                df_tenor = df_date[df_date['Tenor'] == tenor_list[0]]
                x = np.array(df_tenor["Moneyness"])
                y = np.array(df_tenor["Impl_Vol"])
                
                vol_t1 = np.interp(mny_list, x, y)
                #Variance corresponding to tenor t1
                var_t1 = np.square(vol_t1)
               
                #Forward Vol in variance space
                var_t12 = (var_t2 * t2 - var_t1 * t1) / t12
                
                if (np.any((var_t12 < 0))):
                    print ("Calendar Arbitrage observed and corrected by taking absolute value")
                    var_t12 = np.abs(var_t12)
                
                vol_arr = np.sqrt(var_t12)
                
            else:
                vol_arr = np.array([0]) 
                
            if (option_type == "CE" and week != no_of_weeks-1):
                option_value = BSM_call_vec_with_div(stock_vec, opt_strike, r, r_f, vol_arr, dt)
            elif (option_type == "PE" and week != no_of_weeks-1):
                option_value = BSM_put_vec_with_div(stock_vec, opt_strike, r, r_f, vol_arr, dt) 
            elif (option_type == "CE" and week == no_of_weeks-1):
                option_value = np.maximum(stock_vec - opt_strike, 0) 
            else:
                option_value = np.maximum(opt_strike - stock_vec, 0)
    
            option_value = np.asarray(option_value)
            option_value = option_value.reshape(-1, 1) 
            
            y = option_value
            stock_vec = stock_vec.reshape(-1,1)
            x = np.concatenate((np.maximum(stock_vec-call_strikes,0), np.maximum(put_strikes-stock_vec,0)), axis=1) 

            print("SKLearn Linear regression - Lasso")
            lin_model_lasso = Lasso(alpha=1.0).fit(x, y)
            pfl_weights = lin_model_lasso.coef_
            pfl_weights = pfl_weights.reshape(-1)
            cash_weight = lin_model_lasso.intercept_
            print("R-Square: ", lin_model_lasso.score(x, y))
            print("Coefficients:")
            print(pfl_weights)
            print(cash_weight)

#             #Find Optimal Alpha
#             alphas = np.array([-1.0, -0.5, 0.0, 0.25, 0.5, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0])
#             model = LassoCV(alphas=alphas)
#             visualizer = AlphaSelection(model)
#             visualizer.fit(x, y)
#             visualizer.show()    

            # Call Pfl weights
            call_strikes = call_strikes.reshape(-1) 
            ce_dates_list = np.array([sim_start_date.strftime("%d-%b-%Y") for i in range(0, no_of_calls)]).reshape(-1)
            ce_wt_list = pfl_weights[0:no_of_calls]
            ce_dict = {"Date": ce_dates_list, "Strike": call_strikes, "Pfl_weights": ce_wt_list}
            df_ce = pd.DataFrame(ce_dict)
            ce_pfl_weights.append(df_ce)
    
            
            #Put Pfl Weights
            put_strikes = put_strikes.reshape(-1) 
            pe_dates_list = np.array([sim_start_date.strftime("%d-%b-%Y") for i in range(0, no_of_puts)]).reshape(-1)
            pe_wt_list = pfl_weights[no_of_calls:no_ind_vars]
            pe_dict = {"Date": pe_dates_list, "Strike":put_strikes, "Pfl_weights": pe_wt_list}
            df_pe = pd.DataFrame(pe_dict)
            pe_pfl_weights.append(df_pe)
            
            # Cash component
            cash_date = [sim_start_date.strftime("%d-%b-%Y")]
            cash_dict = {"Date":cash_date, "Pfl_weights":cash_weight} 
            df_cash = pd.DataFrame(cash_dict)
            cash_weights.append(df_cash)
            
            
        df_ce_pfl_weights = pd.concat(ce_pfl_weights, axis=0)
        df_pe_pfl_weights = pd.concat(pe_pfl_weights, axis=0)
        df_cash_pfl_weights = pd.concat(cash_weights, axis=0)

#         df_ce_pfl_weights.to_csv(output_path  + "B4A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_CE_Pfl_weights.csv", index = False)
#         df_pe_pfl_weights.to_csv(output_path + "B4A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_PE_Pfl_weights.csv", index = False)
#         df_cash_pfl_weights.to_csv(output_path  + "B4A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_Cash_Pfl_weights.csv", index = False)
    
    return df_ce_pfl_weights, df_pe_pfl_weights, df_cash_pfl_weights

def output_strikes_hedged(prod_type, prod_moneyness, dict_mthly_strikes, stock_ident, output_path):
    list_of_dates = []
    list_of_prod_type = []
    list_of_strikes = []
    for each_month in dict_mthly_strikes.keys():
        list_of_dates.append(each_month)
        list_of_strikes.append(dict_mthly_strikes[each_month])
        list_of_prod_type.append(prod_type)
    dict_hedged_prod = {"Date":list_of_dates, "prod_type":list_of_prod_type, "Strike":list_of_strikes}
    df_hedged_prod = pd.DataFrame(dict_hedged_prod) 
#     df_hedged_prod.to_csv(output_path + "B4A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_strikes_hedged.csv", index = False)
    return(df_hedged_prod)


In [19]:

#Find the monthly strikes of option from mkt data to find the option to be hedged
#Every month, an option is hedged
mthly_expiries_list = get_all_monthly_option_expiries(periods_of_interest, holidays_list)
dict_wkly_expiries_each_month = find_wkly_expries(mthly_expiries_list, holidays_list)

#Load all monthly mkt data
mthly_mkt_data = load_all_mthly_data(mthly_expiries_list, input_data_path, holidays_list, prod_type_lists=["FUT", "CE", "PE"], stock_ident=stock_ident)

#Load weekly strikes of short term options used for hedging monthly options 
dict_ce_wkly_strikes, dict_pe_wkly_strikes, dict_wkly_spots = generate_weekly_strikes(dict_wkly_expiries_each_month, input_data_path, stock_ident = stock_ident)     

#Load monthly strikes of interest as of required moneyness on the begining of month
dict_mthly_strikes = generate_mthly_strikes(mthly_mkt_data, prod_moneyness, prod_type, holidays_list, stock_ident=stock_ident)

#Load weekly IV for Black-Scholes model input and monte-carlo simulation of stock paths
dict_wkly_iv = generate_weekly_iv(dict_wkly_expiries_each_month, dict_wkly_spots, dict_mthly_strikes, prod_type, output_path=output_data_path, atm_ind=0, stock_ident=stock_ident)

#Load weekly risk free interest rates from futures
#This function to be changed based on updates on risk-free-interest - now kept it as cosntant
dict_wkly_ir = generate_weekly_ir(dict_wkly_expiries_each_month, mthly_mkt_data, stock_ident)

#Load Static portfolio weights built by neural network
ce_pfl_wts, pe_pfl_wts, cash_pfl_wts= generate_weekly_static_pfl_weights(no_of_paths, mthly_expiries_list, dict_wkly_expiries_each_month, \
                                                        dict_ce_wkly_strikes, dict_pe_wkly_strikes, dict_wkly_spots, \
                                                        dict_wkly_iv, prod_type, prod_moneyness, output_data_path, \
                                                        option_type = prod_type, no_of_assets=no_of_assets, \
                                                        cor_mat=cor_mat, stock_ident=stock_ident) 
# The strike correspnding to the option that is hedged
df_hedged_prod = output_strikes_hedged(prod_type, prod_moneyness, dict_mthly_strikes, stock_ident, output_data_path)


2019-08-29  Week:  0
['7D', '35D']
SKLearn Linear regression - Lasso
R-Square:  0.9597351589294706
Coefficients:
[-0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  6.50021827e-04  4.58214777e-03  6.86898545e-04  8.11116244e-04
  4.30927493e-04  1.94748733e-04  2.97175600e-05  6.17566492e-05]
[0.00110661]
2019-08-29  Week:  1
['7D', '28D']
SKLearn Linear regression - Lasso
R-Square:  0.9914773993066923
Coefficients:
[-4.73141686e-04 -1.75000575e-04 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  3.76621505e-02  2.64122379e-02
  7.83240753e-03  2.65185529e-03  4.67851125e

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)


R-Square:  1.0
Coefficients:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.]
2019-10-31  Week:  0
['7D', '35D']
SKLearn Linear regression - Lasso


/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.649419616881545, tolerance: 7.543367602390319
  positive)


R-Square:  0.9971583479422699
Coefficients:
[-0.         -0.         -0.00042488 -0.         -0.         -0.
 -0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.05220271  0.02556636  0.01867088  0.00851929  0.0059538
  0.00626484  0.00225809  0.00125451  0.00184559  0.00060044  0.00029993
  0.00367581]
[0.21028173]
2019-10-31  Week:  1
['7D', '28D']
SKLearn Linear regression - Lasso
R-Square:  0.9995008581175487
Coefficients:
[-0.00263791 -0.001249   -0.         -0.          0.          0.
  0.00244173  0.00097045  0.          0.          0.          0.
  0.          0.          0.          0.02366196  0.11837078  0.05430017
  0.03365513  0.03265219  0.02906379  0.00806982  0.02396394  0.01820524
  0.00228599  0.00635317  0.00652143  0.00903574]
[1.27260201]
2019-10-31  Week:  2
['7D', '21D']
SKLearn Linear regression - Lasso
R-Square:  0.999356107995201
Coefficients:
[-0.         -0.00036255 

...

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1532.7801113425803, tolerance: 1290.2295811054196
  positive)


R-Square:  0.9998941917294639
Coefficients:
[-0.09028978 -0.          0.00101922  0.00362602  0.00260018  0.01279224
  0.02330643  0.00540847  0.          0.          0.          0.
  0.00474139  0.0041399   0.00442818  0.00446674  0.00653558  0.00775072
  0.          0.          0.          0.          0.          0.
  0.06563685  0.0803      0.02760031  0.          0.          0.15117431
  0.          0.03396953  0.02417862  0.         -0.00299433  0.04037735
  0.0882536   0.02562609  0.00190385]
[57.65480002]
2020-07-30  Week:  1
['7D', '28D']
SKLearn Linear regression - Lasso


/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 154.56123070241154, tolerance: 106.34208461207855
  positive)


R-Square:  0.9996873858614378
Coefficients:
[-0.01389254 -0.00070519 -0.         -0.          0.          0.00213304
  0.00655303  0.00196108  0.0033336   0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.07849429  0.03805594
  0.03279362  0.02558994  0.01343449  0.0059995   0.00755939  0.01868316
  0.01129912  0.0033047   0.00585115  0.00272565  0.00957388  0.01635229
  0.01039376  0.00874249]
[7.64065866]
2020-07-30  Week:  2
['7D', '21D']
SKLearn Linear regression - Lasso
R-Square:  0.9918124895870888
Coefficients:
[-0.00000000e+00 -0.00000000e+00 -7.61911775e-05 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  1.69862605e-04  2.20760829e-04  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)
